# Week-3 IBM DS Professional Assignment-Part3

Assignment Details & Submission Guidelines

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.
Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)



In [1]:
#Import Dependencies

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('***************Libraries imported*************')

***************Libraries imported*************


In [2]:
#Importing CSV file stored in PART2 https://github.com/shasaankdave/Coursera_Capstone/blob/master/df_toronto_details.csv
url='./df_toronto_details.csv'
toronto_neighbourhoods = pd.read_csv(url,index_col=0)

toronto_neighbourhoods.head()

PostalCode      Borough                              Neighborhood  \
0        M1B  Scarborough                           Malvern / Rouge   
1        M1C  Scarborough  Rouge Hill / Port Union / Highland Creek   
2        M1E  Scarborough       Guildwood / Morningside / West Hill   
3        M1G  Scarborough                                    Woburn   
4        M1H  Scarborough                                 Cedarbrae   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.773136 -79.239476

In [3]:
#Get Toronto,Canada's langitude & longitude using geolocator api
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [4]:
#Plot Toronto city on Map using folium
toronto= folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(toronto_neighbourhoods['Latitude'], toronto_neighbourhoods['Longitude'], toronto_neighbourhoods['Borough'], toronto_neighbourhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto)  

toronto

In [4]:
#Using Foursquare API to get Information about Neighbourhood
# Define Foursquare Credentials and Version
LIMIT = 100

CLIENT_ID = 'ZVLUP1PU41P2XVZN1R15EWMRBMOUU5MA3HTHOYKFKZTHWMYJ' # your Foursquare ID
CLIENT_SECRET = 'ST0MKT1KNGXTV5PXYGRDHNUHA5XRYKTM5V4NGUQG4FHEP245' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZVLUP1PU41P2XVZN1R15EWMRBMOUU5MA3HTHOYKFKZTHWMYJ
CLIENT_SECRET:ST0MKT1KNGXTV5PXYGRDHNUHA5XRYKTM5V4NGUQG4FHEP245


# Explore Toronto Neighbourhood

In [7]:
#Explore Toronto Neighbourhoods for Venues and create data frame
def exploreVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
toronto_venues = exploreVenues(names=toronto_neighbourhoods['Neighborhood'],
                               latitudes=toronto_neighbourhoods['Latitude'],
                               longitudes=toronto_neighbourhoods['Longitude'],
                                  )

Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview / Henry Farm / Oriole
Bayview Village
York Mills / Silver Hills
Willowdale / Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor / Wilson Heights / Downsview North
Northwood Park / York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill / Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West / Riverdale
India Bazaar / The Beaches 

In [10]:
#Venues Data Frame
toronto_venues.head()

Neighborhood  Neighborhood Latitude  \
0                           Malvern / Rouge              43.806686   
1  Rouge Hill / Port Union / Highland Creek              43.784535   
2       Guildwood / Morningside / West Hill              43.763573   
3       Guildwood / Morningside / West Hill              43.763573   
4       Guildwood / Morningside / West Hill              43.763573   

   Neighborhood Longitude                  Venue  Venue Latitude  \
0              -79.194353                Wendy’s       43.807448   
1              -79.160497  Royal Canadian Legion       43.782533   
2              -79.188711      G & G Electronics       43.765309   
3              -79.188711       Big Bite Burrito       43.766299   
4              -79.188711  Enterprise Rent-A-Car       43.764076   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.163085                   Bar  
2       -79.191537     Electronics Store  
3       -79.190720    Mexican Restaurant  
4       -79.193406   Rental Car Location

In [19]:
#Venues Shape
toronto_venues.shape

(2130, 7)

In [12]:
#Group By Neighbourhoods & Venue Category get counts of each category of venue in each neighbourhood
toronto_venues.groupby(['Neighborhood','Venue Category']).count()

Neighborhood Latitude  \
Neighborhood                                       Venue Category                                           
Agincourt                                          Breakfast Spot                                       1   
                                                   Latin American Restaurant                            1   
                                                   Lounge                                               1   
                                                   Skating Rink                                         1   
Alderwood / Long Branch                            Athletics & Sports                                   1   
                                                   Coffee Shop                                          1   
                                                   Gym                                                  1   
                                                   Pharmacy                                             1   
                                                   Pizza Place                                          2   
                                                   Pub                                                  1   
                                                   Sandwich Place                                       1   
                                                   Skating Rink                                         1   
Bathurst Manor / Wilson Heights / Downsview North  Bank                                                 2   
                                                   Bridal Shop                                          1   
                                                   Coffee Shop                                          2   
                                                   Deli / Bodega                                        1   
                                                   Diner                                                1   
                                                   Fried Chicken Joint                                  1   
                                                   Gas Station                                          1   
                                                   Ice Cream Shop                                       1   
                                                   Middle Eastern Restaurant                            1   
                                                   Pet Store                                            1   
                                                   Pharmacy                                             1   
                                                   Pizza Place                                          1   
                                                   Restaurant                                           1   
                                                   Sandwich Place                                       1   
                                                   Shopping Mall                                        1   
                                                   Supermarket                                          1   
                                                   Sushi Restaurant                                     1   
                                                   Video Store                                          1   
Bayview Village                                    Bank                                                 1   
                                                   Café                                                 1   
                                                   Chinese Restaurant                                   1   
                                                   Japanese Restaurant                                  1   
Bedford Park / Lawrence Manor East                 American Restaurant                                  1   
                                                   

In [23]:
#There are 271 categories of Venue
Venue_categories=toronto_venues['Venue Category'].unique()
Venue_categories.shape

(271,)

In [14]:
#One hot coding  is a technique to create matrix to add 1 for exists and 0 for doesnot exists.

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
#So by dataframe below we can infer:For Example:Malvern / Rouge neighbourhood doesnot have yoga studio.

Yoga Studio  Accessories Store  Airport  Airport Food Court  Airport Gate  \
0            0                  0        0                   0             0   
1            0                  0        0                   0             0   
2            0                  0        0                   0             0   
3            0                  0        0                   0             0   
4            0                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  \
0             0         0            0           0                    0   
1             0         0            0           0                    0   
2             0         0            0           0                    0   
3             0         0            0           0                    0   
4             0         0            0           0                    0   

   Asian Restaurant  Athletics & Sports  Auto Workshop  BBQ Joint  Baby Store  \
0                 0                   0              0          0           0   
1                 0                   0              0          0           0   
2                 0                   0              0          0           0   
3                 0                   0              0          0           0   
4                 0                   0              0          0           0   

   Bagel Shop  Bakery  Bank  Bar  Baseball Field  Baseball Stadium  \
0           0       0     0    0               0                 0   
1           0       0     0    1               0                 0   
2           0       0     0    0               0                 0   
3           0       0     0    0               0                 0   
4           0       0     0    0               0                 0   

   Basketball Court  Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  \
0                 0                   0      0                0         0   
1                 0                   0      0                0         0   
2                 0                   0      0                0         0   
3                 0                   0      0                0         0   
4                 0                   0      0                0         0   

   Beer Store  Belgian Restaurant  Bike Shop  Bistro  Boat or Ferry  \
0           0                   0          0       0              0   
1           0                   0          0       0              0   
2           0                   0          0       0              0   
3           0                   0          0       0              0   
4           0                   0          0       0              0   

   Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  Brewery  \
0          0         0                     0               0        0   
1          0         0                     0               0        0   
2          0         0                     0               0        0   
3          0         0                     0               0        0   
4          0         0                     0               0        0   

   Bridal Shop  Bubble Tea Shop  Building  Burger Joint  Burrito Place  \
0            0                0         0             0              0   
1            0                0         0             0              0   
2            0                0         0             0              0   
3            0                0         0             0              0   
4

In [15]:
#Frquence of Occurences
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio  \
0                                           Agincourt     0.000000   
1                             Alderwood / Long Branch     0.000000   
2   Bathurst Manor / Wilson Heights / Downsview North     0.000000   
3                                     Bayview Village     0.000000   
4                  Bedford Park / Lawrence Manor East     0.000000   
5                                         Berczy Park     0.000000   
6                        Birch Cliff / Cliffside West     0.000000   
7      Brockton / Parkdale Village / Exhibition Place     0.000000   
8               Business reply mail Processing CentrE     0.000000   
9   CN Tower / King and Spadina / Railway Lands / ...     0.000000   
10                                Caledonia-Fairbanks     0.000000   
11              Canada Post Gateway Processing Centre     0.000000   
12                                          Cedarbrae     0.000000   
13                                 Central Bay Street     0.015385   
14                                           Christie     0.000000   
15                               Church and Wellesley     0.027778   
16          Clarks Corners / Tam O'Shanter / Sullivan     0.000000   
17  Cliffside / Cliffcrest / Scarborough Village West     0.000000   
18                    Commerce Court / Victoria Hotel     0.000000   
19                                         Davisville     0.000000   
20                                   Davisville North     0.000000   
21  Del Ray / Mount Dennis / Keelsdale and Silvert...     0.000000   
22                                          Don Mills     0.000000   
23  Dorset Park / Wexford Heights / Scarborough To...     0.000000   
24                                          Downsview     0.000000   
25                      Dufferin / Dovercourt Village     0.000000   
26                                       East Toronto     0.000000   
27  Eringate / Bloordale Gardens / Old Burnhamthor...     0.000000   
28                     Fairview / Henry Farm / Oriole     0.000000   
29            First Canadian Place / Underground city     0.000000   
30                           Forest Hill North & West     0.000000   
31                           Garden District, Ryerson     0.000000   
32                                          Glencairn     0.000000   
33                  Golden Mile / Clairlea / Oakridge     0.000000   
34                Guildwood / Morningside / West Hill     0.000000   
35  Harbourfront East / Union Station / Toronto Is...     0.000000   
36                     High Park / The Junction South     0.000000   
37                                  Hillcrest Village     0.000000   
38                                      Humber Summit     0.000000   
39                                  Humberlea / Emery     0.000000   
40                                 Humewood-Cedarvale     0.000000   
41                    India Bazaar / The Beaches West     0.000000   
42      Kennedy Park / Ionview / East Birchmount Park     0.000000   
43        Kensington Market / Chinatown / Grange Park     0.000000   
44  Kingsview Village / St. Phillips / Martin Grov...     0.000000   
45                  Lawrence Manor / Lawrence Heights     0.000000   
46                                      Lawrence Park     0.000000   
47                                            Leaside     0.000000   
48                          Little Portugal / Trinity     0.023810   
49                                    Malvern / Rouge     0.000000   
50  Milliken / Agincourt North / Steeles East / L'...     0.000000   
51  Mimico NW / The Queensway West / South of Bloo...     0.000000   
52                       Moore Park / Summerhill East     0.000000   
53     New Toronto / Mimico South / Humber Bay Shores     0.000000   
54         North Park / Maple Leaf Park / Upwood Park     0.000000   
55                                 North Toronto West     0.050000   
56                                          Northwe

In [16]:
#Lets print top 2 venues in each neighbourhood
num_top_venues = 2

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25


----Alderwood / Long Branch----
                venue  freq
0         Pizza Place  0.22
1  Athletics & Sports  0.11


----Bathurst Manor / Wilson Heights / Downsview North----
         venue  freq
0  Coffee Shop   0.1
1         Bank   0.1


----Bayview Village----
  venue  freq
0  Café  0.25
1  Bank  0.25


----Bedford Park / Lawrence Manor East----
         venue  freq
0  Coffee Shop  0.09
1   Restaurant  0.09


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05


----Birch Cliff / Cliffside West----
             venue  freq
0             Café  0.25
1  College Stadium  0.25


----Brockton / Parkdale Village / Exhibition Place----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09


----Business reply mail Processing CentrE----
                  venue  freq
0  Gym / Fitness Center  0.06
1         Auto Worksh

           venue  freq
0  Grocery Store  0.22
1    Pizza Place  0.11


----St. James Town----
         venue  freq
0  Coffee Shop  0.06
1         Café  0.06


----St. James Town / Cabbagetown----
  venue  freq
0  Park  0.07
1  Café  0.07


----Steeles West / L'Amoreaux West----
                  venue  freq
0  Fast Food Restaurant  0.13
1    Chinese Restaurant  0.13


----Stn A PO Boxes----
                venue  freq
0         Coffee Shop  0.08
1  Italian Restaurant  0.05


----Studio District----
         venue  freq
0         Café  0.10
1  Coffee Shop  0.08


----Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park----
         venue  freq
0  Coffee Shop  0.12
1          Pub  0.12


----The Annex / North Midtown / Yorkville----
            venue  freq
0  Sandwich Place  0.14
1            Café  0.14


----The Beaches----
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25


----The Danforth West / Riverdale----
                venue  freq

In [17]:
#Get Most favorite Venues 

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood  \
0                                          Agincourt   
1                            Alderwood / Long Branch   
2  Bathurst Manor / Wilson Heights / Downsview North   
3                                    Bayview Village   
4                 Bedford Park / Lawrence Manor East   

       1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0  Latin American Restaurant          Skating Rink                Lounge   
1                Pizza Place                   Gym           Coffee Shop   
2                Coffee Shop                  Bank      Sushi Restaurant   
3                       Bank    Chinese Restaurant   Japanese Restaurant   
4             Sandwich Place    Italian Restaurant            Restaurant   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0        Breakfast Spot            Donut Shop                 Diner   
1        Sandwich Place    Athletics & Sports                   Pub   
2           Pizza Place           Bridal Shop        Sandwich Place   
3                  Café         Women's Store      Doner Restaurant   
4           Coffee Shop             Juice Bar           Pizza Place   

  7th Most Common Venue    8th Most Common Venue 9th Most Common Venue  \
0        Discount Store      Distribution Center               Dog Run   
1          Skating Rink                 Pharmacy      Department Store   
2                 Diner               Restaurant        Ice Cream Shop   
3                 Diner           Discount Store   Distribution Center   
4              Boutique  Comfort Food Restaurant          Liquor Store   

        10th Most Common Venue  
0             Doner Restaurant  
1  Eastern European Restaurant  
2                Deli / Bodega  
3                      Dog Run  
4            Indian Restaurant

In [19]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [31]:
#add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels1', kmeans.labels_)

#toronto_merged = toronto_neighbourhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head() # check the last columns!

PostalCode      Borough                              Neighborhood  \
0        M1B  Scarborough                           Malvern / Rouge   
1        M1C  Scarborough  Rouge Hill / Port Union / Highland Creek   
2        M1E  Scarborough       Guildwood / Morningside / West Hill   
3        M1G  Scarborough                                    Woburn   
4        M1H  Scarborough                                 Cedarbrae   

    Latitude  Longitude  Cluster Labels1  Cluster Labels  \
0  43.806686 -79.194353              0.0             0.0   
1  43.784535 -79.160497              1.0             1.0   
2  43.763573 -79.188711              1.0             1.0   
3  43.770992 -79.216917              1.0             1.0   
4  43.773136 -79.239476              1.0             1.0   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0  Fast Food Restaurant          Dessert Shop           Event Space   
1                   Bar         Women's Store            Donut Shop   
2   Rental Car Location          Intersection        Breakfast Spot   
3           Coffee Shop     Korean Restaurant         Women's Store   
4  Caribbean Restaurant   Fried Chicken Joint                  Bank   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0  Ethiopian Restaurant   Empanada Restaurant     Electronics Store   
1                 Diner        Discount Store   Distribution Center   
2     Electronics Store    Mexican Restaurant                  Bank   
3            Donut Shop                 Diner        Discount Store   
4    Athletics & Sports       Thai Restaurant           Gas Station   

         7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
0  Eastern European Restaurant   Dumpling Restaurant             Drugstore   
1                      Dog Run      Doner Restaurant             Drugstore   
2               Medical Center        Discount Store   Distribution Center   
3          Distribution Center               Dog Run      Doner Restaurant   
4                       Bakery      Hakka Restaurant   Dumpling Restaurant   

  10th Most Common Venue  
0             Donut Shop  
1         Farmers Market  
2                Dog Run  
3    Dumpling Restaurant  
4              Drugstore

In [90]:
toronto_merged = toronto_merged[toronto_merged['Cluster Labels'].notna()]

In [95]:
# create map with cluster color_coding
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]
color_code = ['green','yellow','beige','red','blue']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if cluster != 'Nan':
        val=int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=color_code[val],
        fill=True,
        fill_color=rainbow[val-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [92]:
#Test Cluster Labels
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels1  Cluster Labels  \
1         Scarborough              1.0             1.0   
2         Scarborough              1.0             1.0   
3         Scarborough              1.0             1.0   
4         Scarborough              1.0             1.0   
6         Scarborough              1.0             1.0   
7         Scarborough              1.0             1.0   
8         Scarborough              1.0             1.0   
9         Scarborough              1.0             1.0   
10        Scarborough              1.0             1.0   
11        Scarborough              1.0             1.0   
12        Scarborough              1.0             1.0   
13        Scarborough              1.0             1.0   
15        Scarborough              1.0             1.0   
17         North York              1.0             1.0   
18         North York              1.0             1.0   
19         North York              1.0             1.0   
20         North York              1.0             1.0   
21         North York              1.0             1.0   
22         North York              1.0             1.0   
24         North York              1.0             1.0   
26         North York              1.0             1.0   
27         North York              1.0             1.0   
28         North York              1.0             1.0   
29         North York              1.0             1.0   
30         North York              1.0             1.0   
31         North York              1.0             1.0   
32         North York              1.0             1.0   
33         North York              1.0             1.0   
34         North York              1.0             1.0   
35          East York              1.0             1.0   
36          East York              1.0             1.0   
37       East Toronto              1.0             1.0   
38          East York              1.0             1.0   
39          East York              1.0             1.0   
41       East Toronto              1.0             1.0   
42       East Toronto              1.0             1.0   
43       East Toronto              1.0             1.0   
45    Central Toronto              1.0             1.0   
46    Central Toronto              1.0             1.0   
47    Central Toronto              1.0             1.0   
49    Central Toronto              1.0             1.0   
51   Downtown Toronto              1.0             1.0   
52   Downtown Toronto              1.0             1.0   
53   Downtown Toronto              1.0             1.0   
54   Downtown Toronto              1.0             1.0   
55   Downtown Toronto              1.0             1.0   
56   Downtown Toronto              1.0             1.0   
57   Downtown Toronto              1.0             1.0   
58   Downtown Toronto              1.0             1.0   
59   Downtown Toronto              1.0             1.0   
60   Downtown Toronto              1.0             1.0   
61   Downtown Toronto              1.0             1.0   
62         North York              1.0             1.0   
63    Central Toronto              1.0             1.0   
64    Central Toronto              1.0             1.0   
65    Central Toronto              1.0             1.0   
66   Downtown Toronto              1.0             1.0   
67   Downtown Toronto              1.0             1.0   
68   Downtown Toronto              1.0             1.0   
69   Downtown Toronto              1.0             1.0   
70   Downtown Toronto              1.0             1.0   
71         North York              1.0             1.0   
73               York              1.0             1.0   
75   Downtown Toronto              1.0             1.0   
76       West Toronto              1.0             1.0   
77       West Toronto              1.0             1.0   
78       West Toronto              1.0             1.0   
80               York              1.0             1.0   
81          

# Cluster 0 is the biggest